In [1]:
#!pip install transformers datasets accelerate huggingface_hub

In [2]:
# !pip install -U trl

In [3]:
import os
import json
from getpass import getpass
from huggingface_hub import login

In [4]:
token = getpass(prompt="enter huggingface token")
login(token=token)

enter huggingface token ········


In [5]:
dir_name = "data"

if os.path.isfile(dir_name + '/topics.json'):
    with open(dir_name + '/topics.json') as t:
        topics = json.load(t)

    topics = topics.keys()
    tfiles = [dir_name + f"""/{"-".join(t.lower().split(' '))}.json""" for t in topics]
    tfiles = [tf for tf in tfiles if os.path.isfile(tf)]

    data = []
    for tf in tfiles:
        with open(tf) as t:
            data.extend(json.load(t))

    print(f'there are {len(data)} rows in the dataset')

there are 180 rows in the dataset


In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset

In [7]:
dataset = load_dataset("json", data_files=tfiles)

In [8]:
dataset = dataset["train"].train_test_split(test_size=0.3, seed=42)

In [9]:
def preprocess(row):
    output_texts = []
    for i in range(len(row['question'])):
        text = f'<question>\n{row["question"][i]}\n</question>\n\n<documents>\n{row["documents"][i]}\n</documents>\n\n<answer>\n{row["answer"][i]}\n</answer>'
        output_texts.append(text)
    return output_texts

In [12]:
from transformers import BitsAndBytesConfig

model_id = "meta-llama/Llama-3.2-1B"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config)

`low_cpu_mem_usage` was None, now default to True since model is quantized.


In [13]:
from peft import LoraConfig, get_peft_model

# Define LoRA Configuration
lora_config = LoraConfig(
    r=4,               # Low-rank dimension
    lora_alpha=32,     # Scaling factor
    lora_dropout=0.1,  # Dropout for regularization
    bias="none",       # No bias training
    task_type="CAUSAL_LM"  # Task type: Causal Language Modeling
)

# Wrap the model with LoRA
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 425,984 || all params: 1,236,240,384 || trainable%: 0.0345


In [14]:
# model = model.to(torch.device('cpu'))

In [15]:
tokenizer.add_special_tokens({"pad_token": "[PAD]"})
model.resize_token_embeddings(len(tokenizer))

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Embedding(128257, 2048)

In [16]:
from trl import SFTConfig, SFTTrainer, DataCollatorForCompletionOnlyLM

collator = DataCollatorForCompletionOnlyLM(response_template=">\n\n<answer>\n", tokenizer=tokenizer)

trainer = SFTTrainer(
    model,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    args=SFTConfig(output_dir="./tmp",
                   max_seq_length=1536,
                   logging_steps=10,
                   num_train_epochs=30,
                   do_eval=True,
                   per_device_train_batch_size=2,
                   report_to="none"),
    formatting_func=preprocess,
    data_collator=collator,
)

trainer.train()

/run/nvme/job_26995403/tmp/ipykernel_305634/552167012.py:5: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


Applying formatting function to train dataset:   0%|          | 0/126 [00:00<?, ? examples/s]

Converting train dataset to ChatML:   0%|          | 0/126 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/126 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/126 [00:00<?, ? examples/s]

...

[2025-03-07 12:38:43,517] [INFO] [real_accelerator.py:219:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/opt/rh/gcc-toolset-12/root/usr/libexec/gcc/x86_64-redhat-linux/12/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlvsym'
/opt/rh/gcc-toolset-12/root/usr/libexec/gcc/x86_64-redhat-linux/12/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlopen'
/opt/rh/gcc-toolset-12/root/usr/libexec/gcc/x86_64-redhat-linux/12/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlclose'
/opt/rh/gcc-toolset-12/root/usr/libexec/gcc/x86_64-redhat-linux/12/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlerror'
/opt/rh/gcc-toolset-12/root/usr/libexec/gcc/x86_64-redhat-linux/12/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlsym'
collect2: error: ld returned 1 exit status
/usr/local/lib64/python3.11/site-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


Step,Training Loss
10,1.587400
20,1.513300
30,1.773000
40,1.378500
50,1.562200
60,1.399100
70,1.509700
80,1.334800
90,1.309000
100,1.175200


/usr/local/lib/python3.11/site-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.11/site-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.11/site-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.11/site-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


TrainOutput(global_step=1890, training_loss=0.8816093628999417, metrics={'train_runtime': 1416.3151, 'train_samples_per_second': 2.669, 'train_steps_per_second': 1.334, 'total_flos': 1.998066685766861e+16, 'train_loss': 0.8816093628999417})

In [17]:
prompt = """
<question>
How should we treat animals?
</question>

<documents>
Document 1: But it’s been thirty years and vegans are still less than 10% of the population.If you genuinely care about animal suffering, you have to admit that, and say, ‘what else can we do to ease animal suffering?’”After Singer’s talk, I began thinking through the consequences of his morality.A question occurred to me: “Should we also stop animals from eating each other?” I was sure others had made such arguments as *reductio ad absurdums* of vegetarianism, but I thought I might be the first to be genuinely interested in it from a moral perspective.
---
Document 2: They didn’t even pretend to make an argument; it was entirely mockery.Watching this, I couldn’t help but realize there was a powerful logical argument at the core of the animal rights groups: animals should be treated much the same way humans are — their lives should be respected, their pain minimized, etc.Make this one simple change to your system of morality and everything else falls into place.
---
Document 3: But I praise people for the good things they do and condemn them for the bad ones.”A final question raised the incrementalism versus revolutionism debate common to all left-wing social movements.Should one really worry about animal treatment when the animals were still going to be killed?Pinger said the answer was undoubtedly yes.
---
Document 4: They didn’t even pretend to make an argument; it was entirely mockery.Watching this, I couldn’t help but realize there was a powerful logical argument at the core of the animal rights groups: animals should be treated much the same way humans are — their lives should be respected, their pain minimized, etc.Make this one simple change to your system of morality and everything else falls into place.
---
Document 5: But I praise people for the good things they do and condemn them for the bad ones.”A final question raised the incrementalism versus revolutionism debate common to all left-wing social movements.Should one really worry about animal treatment when the animals were still going to be killed?Pinger said the answer was undoubtedly yes.
---
Document 6: But it’s been thirty years and vegans are still less than 10% of the population.If you genuinely care about animal suffering, you have to admit that, and say, ‘what else can we do to ease animal suffering?’”After Singer’s talk, I began thinking through the consequences of his morality.A question occurred to me: “Should we also stop animals from eating each other?” I was sure others had made such arguments as *reductio ad absurdums* of vegetarianism, but I thought I might be the first to be genuinely interested in it from a moral perspective.
</documents>

<answer>
""".strip()

In [27]:
inputs = tokenizer(prompt, return_tensors="pt").to('cuda')

# Generate output (adjust parameters as needed)
outputs = model.generate(
    **inputs,
    max_length=1536,      # Maximum length of generated sequence
    # do_sample=True,      # Use sampling instead of greedy decoding
    temperature=0.8,     # Controls randomness: lower is less random
    top_p=0.9            # Use nucleus sampling to limit choices to a probability mass
)

# Decode and print the generated text
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(len(outputs))
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


1
<question>
How should we treat animals?
</question>

<documents>
Document 1: But it’s been thirty years and vegans are still less than 10% of the population.If you genuinely care about animal suffering, you have to admit that, and say, ‘what else can we do to ease animal suffering?’”After Singer’s talk, I began thinking through the consequences of his morality.A question occurred to me: “Should we also stop animals from eating each other?” I was sure others had made such arguments as *reductio ad absurdums* of vegetarianism, but I thought I might be the first to be genuinely interested in it from a moral perspective.
---
Document 2: They didn’t even pretend to make an argument; it was entirely mockery.Watching this, I couldn’t help but realize there was a powerful logical argument at the core of the animal rights groups: animals should be treated much the same way humans are — their lives should be respected, their pain minimized, etc.Make this one simple change to your system of mor